In [1]:
# 한국어 적용과 시각화설정
import matplotlib as mpl
import matplotlib.pyplot as plt
 
%config InlineBackend.figure_format = 'retina'
 
import matplotlib.font_manager as fm
fontpath = '/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf'
font = fm.FontProperties(fname=fontpath, size=9)
plt.rc('font', family='NanumBarunGothic') 
mpl.font_manager.findfont(font)

import pandas as pd
import tensorflow as tf


print(pd.__version__)
print(tf.__version__)
print(mpl.__version__)

print("완료!")

1.3.3
2.6.0
3.4.3
완료!


# 데이터 준비
프로젝트 라이브러리

In [2]:
import tensorflow as tf
import numpy as np

from sklearn.model_selection import train_test_split

import matplotlib.ticker as ticker
import matplotlib.pyplot as plt

import time
import re
import os
import io

print(tf.__version__)

2.6.0


# 데이터 전처리 
## 1. 정제


In [4]:
path_to_ko = os.path.join(current_dir, "korean-english-park.train.ko")
path_to_en = os.path.join(current_dir, "korean-english-park.train.en")


In [45]:
import tarfile
from konlpy.tag import Mecab

# 파일 경로 설정
current_dir = os.getcwd()
path_to_ko = os.path.join(current_dir, "korean-english-park.train.ko")
path_to_en = os.path.join(current_dir, "korean-english-park.train.en")

# 파일 내용 읽기
with open(path_to_ko, 'r', encoding='utf-8') as f:
    kor_data = f.read().splitlines()

with open(path_to_en, 'r', encoding='utf-8') as f:
    eng_data = f.read().splitlines()

# 데이터 크기와 예시 출력
print("한국어 데이터 크기:", len(kor_data))
print("영어 데이터 크기:", len(eng_data))
print("한국어 예시 데이터:")
for sen in kor_data[:10]:
    print(">>", sen)

print("영어 예시 데이터:")
for sen in eng_data[:10]:
    print(">>", sen)

# 중복 데이터 제거
cleaned_corpus = list(set(zip(kor_data, eng_data)))

# 전처리 함수 정의
mecab = Mecab()

def preprocess_sentence_korean(sentence, s_token=False, e_token=False):
    sentence = sentence.strip()
    
    # 특수 문자 제거
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence)
    sentence = re.sub(r'[" "]+', " ", sentence)
    
    # 한글과 영어를 제외한 문자 제거
    sentence = re.sub(r"[^가-힣a-zA-Z?.!,]+", " ", sentence)
    
    sentence = sentence.strip()

    if s_token:
        sentence = '<start> ' + sentence

    if e_token:
        sentence += ' <end>'
    
    return sentence

# 병렬 데이터 전처리
kor_sentences = []
eng_sentences = []

for kor, eng in cleaned_corpus:
    kor_sentences.append(preprocess_sentence_korean(kor))
    eng_sentences.append(preprocess_sentence_korean(eng, s_token=True, e_token=True))

print("전처리 후 한국어 예시 데이터:", kor_sentences[:5])
print("전처리 후 영어 예시 데이터:", eng_sentences[:5])

# 길이가 40 이하인 데이터 필터링
def filter_long_sentences(kor, eng, max_len=40):
    filtered_kor = []
    filtered_eng = []
    
    for k, e in zip(kor, eng):
        if len(k.split()) <= max_len and len(e.split()) <= max_len:
            filtered_kor.append(k)
            filtered_eng.append(e)
    
    return filtered_kor, filtered_eng

filtered_kor, filtered_eng = filter_long_sentences(kor_sentences, eng_sentences)

print("필터링 후 한국어 데이터 크기:", len(filtered_kor))
print("필터링 후 영어 데이터 크기:", len(filtered_eng))
from konlpy.tag import Mecab

한국어 데이터 크기: 94123
영어 데이터 크기: 94123
한국어 예시 데이터:
>> 개인용 컴퓨터 사용의 상당 부분은 "이것보다 뛰어날 수 있느냐?"
>> 모든 광마우스와 마찬가지 로 이 광마우스도 책상 위에 놓는 마우스 패드를 필요로 하지 않는다.
>> 그러나 이것은 또한 책상도 필요로 하지 않는다.
>> 79.95달러하는 이 최첨단 무선 광마우스는 허공에서 팔목, 팔, 그외에 어떤 부분이든 그 움직임에따라 커서의 움직임을 조절하는 회전 운동 센서를 사용하고 있다.
>> 정보 관리들은 동남 아시아에서의 선박들에 대한 많은 (테러) 계획들이 실패로 돌아갔음을 밝혔으며, 세계 해상 교역량의 거의 3분의 1을 운송하는 좁은 해로인 말라카 해협이 테러 공격을 당하기 쉽다고 경고하고 있다.
>> 이 지역에 있는 미국 선박과 상업용 선박들에 대한 알카에다의 (테러) 시도 중 여러 건이 실패했다는 것을 알게 된 후에, 전문가들은 테러 조직이 여전히 세계 경제에 타격을 입히려 한다고 경고하고 있으며, 동남 아시아에 있는 세계 경제의 주요 통로가 위험에 처해 있다고 그들은 생각하고 있다.
>> 국립 과학 학회가 발표한 새 보고서에따르면, 복잡한 임무를 수행해야 하는 군인들이나 보다 오랜 시간 동안 경계를 늦추지 않고 있기 위해 도움이 필요한 군인들에게 카페인이 반응 시간을 증가시키고 임무 수행 능력을 향상시키는데 도움이 된다고 한다.
>> 이 보고서에따르면, "특히, 군사 작전에서 생사가 걸린 상황이 될 수도 있는 반응 속도와 시각 및 청각의 경계 상태를 유지시키기 위해 카페인이 사용될 수도 있다." 고 한다.
>> "결정적인 순간에 그들의 능력을 증가시켜 줄 그 무엇이 매우 중요합니다."
>> 연구가들이 이미 커피 대체품으로서 음식 대용 과자나 껌에 카페인을 첨가하는 방법을 연구하고 있다고 Archibald는 말했다.
영어 예시 데이터:
>> Much of personal computing is about "can you top this?"
>> so a mentio

## 2. 토큰화
### 정제된 텍스트를 아래 tokenize() 함수를 사용해 토큰화하고 텐서로 변환하세요. 그리고 변환된 텐서를 80%의 훈련 데이터와 20%의 검증 데이터로 분리하세요! (Tokenizer의 단어 수는 자유롭게 진행하세요!)

In [60]:
from konlpy.tag import Mecab

def enc_tokenize(corpus):
    tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=15000,
                                                      filters='')
    mecab = Mecab()
    tokenized_corpus = []
    
    for sen in corpus:
        tokenized_sen = mecab.morphs(sen) #토큰화
        tokenized_corpus.append(tokenized_sen)
        
    tokenizer.fit_on_texts(tokenized_corpus)
    
    sequences = tokenizer.texts_to_sequences(tokenized_corpus)  # 정수 인코딩
    padded_sequences = tf.keras.preprocessing.sequence.pad_sequences(sequences, 
                                                                     padding='post')  # 패딩

    return padded_sequences, tokenizer


#영어 토큰화 함수
def dec_tokenize(corpus):
    #토크나이저 객체생성, filters = ' ' :특수문자 필터링 비활성황 
    tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=15000,
                                                      filters='')
    
    #텍스트 말뭉치(corpus)에 등장하는 모든 단어를 분석하고 각 단어에 고유한 정수 인덱스를 부여
    tokenizer.fit_on_texts(corpus)

    #텍스트 말뭉치를 정수 시퀀스로 변환
    tensor = tokenizer.texts_to_sequences(corpus)

    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, 
                                                           padding='post')

    return tensor, tokenizer

enc_train, enc_tokenizer = enc_tokenize(kor_sentences)
dec_train, dec_tokenizer = dec_tokenize(eng_sentences)

# 모델설계
### Encoder는 모든 Time-Step의 Hidden State를 출력으로 갖고, Decoder는 Encoder의 출력과 Decoder의 t-1 Step의 Hidden State로 Attention을 취하여 t Step의 Hidden State를 만들어 냅니다.Decoder에서 t Step의 단어로 예측된 것을 실제 정답과 대조해 Loss를 구하고, 생성된 t Step의 Hidden State는 t+1 Step의 Hidden State를 만들기 위해 다시 Decoder에 전달됩니다.여기서 't=1 일 때의 Hidden State는 어떻게 정의할 것인가?' 가 궁금하실 수 있는데요, 일반적으로 Encoder의 Final State를 Hidden State로 사용합니다.

In [48]:
class BahdanauAttention(tf.keras.layers.Layer):
    def __init__(self, units):
        super(BahdanauAttention, self).__init__()
        self.w_dec = tf.keras.layers.Dense(units)
        self.w_enc = tf.keras.layers.Dense(units)
        self.w_com = tf.keras.layers.Dense(1)
    
    def call(self, h_enc, h_dec):
        # h_enc shape: [batch x length x units]
        # h_dec shape: [batch x units]

        h_enc = self.w_enc(h_enc)
        h_dec = tf.expand_dims(h_dec, 1)
        h_dec = self.w_dec(h_dec)

        score = self.w_com(tf.nn.tanh(h_dec + h_enc))
        
        attn = tf.nn.softmax(score, axis=1)

        context_vec = attn * h_enc
        context_vec = tf.reduce_sum(context_vec, axis=1)

        return context_vec, attn


In [51]:


class Encoder(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, enc_units, dropout_rate=0.5):
        super(Encoder, self).__init__()
        
        self.enc_units = enc_units
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = tf.keras.layers.GRU(enc_units,
                                       return_sequences=True,
                                       return_state=True,
                                       recurrent_initializer='glorot_uniform')
        self.dropout = tf.keras.layers.Dropout(dropout_rate)
        
    def call(self, x):
        x = self.embedding(x)
        x = self.dropout(x)
        output, state = self.gru(x)
        return output, state


In [54]:


class Decoder(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, dec_units, dropout_rate=0.5):
        super(Decoder, self).__init__()
        self.dec_units = dec_units
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = tf.keras.layers.GRU(dec_units,
                                       return_sequences=True,
                                       return_state=True,
                                       recurrent_initializer='glorot_uniform')
        self.fc = tf.keras.layers.Dense(vocab_size)
        self.dropout = tf.keras.layers.Dropout(dropout_rate)
        self.attention = BahdanauAttention(self.dec_units)

    def call(self, x, h_dec, enc_out):
        context_vec, attn = self.attention(enc_out, h_dec)

        x = self.embedding(x)
        x = tf.concat([tf.expand_dims(context_vec, 1), x], axis=-1)
        
        x, h_dec = self.gru(x)
        x = self.dropout(x)
        x = tf.reshape(x, (-1, x.shape[2]))
        x = self.fc(x)

        return x, h_dec, attn




In [56]:
BATCH_SIZE     = 64
SRC_VOCAB_SIZE = len(kor_tokenizer.index_word) + 1
TGT_VOCAB_SIZE = len(eng_tokenizer.index_word) + 1

units         = 1024
embedding_dim = 512

encoder = Encoder(SRC_VOCAB_SIZE, embedding_dim, units)
decoder = Decoder(TGT_VOCAB_SIZE, embedding_dim, units)

# sample input
sequence_len = 30

sample_enc = tf.random.uniform((BATCH_SIZE, sequence_len))
sample_output = encoder(sample_enc)

print ('Encoder Output:', sample_output.shape)

sample_state = tf.random.uniform((BATCH_SIZE, units))

sample_logits, h_dec, attn = decoder(tf.random.uniform((BATCH_SIZE, 1)),
                                     sample_state, sample_output)

print ('Decoder Output:', sample_logits.shape)
print ('Decoder Hidden State:', h_dec.shape)
print ('Attention:', attn.shape)

AttributeError: 'tuple' object has no attribute 'shape'

# Optimizer & Loss

In [35]:
optimizer = tf.keras.optimizers.Adam()
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction='none')

def loss_function(real, pred):
    mask = tf.math.logical_not(tf.math.equal(real, 0))
    loss = loss_object(real, pred)
    mask = tf.cast(mask, dtype=loss.dtype)
    loss *= mask
    return tf.reduce_mean(loss)


### Optimizer는 모델이 학습할 때에 정답을 찾아가는 방법

### fit() 함수를 사용할 수 없는 이유는 바로 Loss 함수 때문

### Encoder-Decoder 구조는 학습 과정이 일반적이지 않으므로 직접 Loss를 커스텀해서 사용

SparseCategoricalCrossentropy() 함수는 모델이 출력한 확률 분포와 (One-hot이 아닌) 정수 인덱스 답안을 비교해 Cross Entropy값을 구해줍니다. CategoricalCrossentropy()라면 [ 0.1, 0.2, 0.7 ] 과 One-hot 인코딩된 라벨 [0, 0, 1] 을 비교하여 점수를 채점하겠지만, SparseCategoricalCrossentropy() 함수라면 [ 0.1, 0.2, 0.7 ] 과 정수 인덱스 답안 2 를 비교하여 점수를 채점하는 거죠. from_logits 는 확률 분포가 Softmax를 거쳐서 들어오는지, 모델의 출력값 그대로 들어오는지를 결정합니다. 우리는 True 로 줬으니 모델의 출력값을 그대로 전달하면 됩니다!

### 패딩에 대한 처리를 해주지 않으면 "PAD" 토큰만을 생성할 확률이 굉장히 높아집니다.

mask 는 정답지에서 'PAD' 토큰을 찾아내어 그 부분에 대한 Loss는 구하지 않도록 하는 역할을 해주죠. equal() 함수에 정확히는 0 이 아닌 <PAD> 토큰의 인덱스를 전달하는 것이 맞지만 대부분의 경우는 0으로 패딩되기 때문에 편의상 0을 전달하여 처리

# 훈련

train_step()은 학습에 필요한 것을 모두 가져가 Loss를 계산한 후 반환하는 함수

In [36]:
@tf.function
def train_step(input, target, encoder, decoder, optimizer, eng_tokenizer):
    loss = 0
    with tf.GradientTape() as tape:
        # 인코더 처리
        enc_output, enc_hidden = encoder(input, encoder.initialize_hidden_state())

        dec_hidden = enc_hidden
        dec_input = tf.expand_dims([eng_tokenizer.word_index['<start>']] * target.shape[0], 1)

        # 디코더 처리
        for t in range(1, target.shape[1]):
            predictions, dec_hidden, _ = decoder(dec_input, dec_hidden, enc_output)
            loss += loss_function(target[:, t], predictions)
            dec_input = tf.expand_dims(target[:, t], 1)

    batch_loss = (loss / int(target.shape[1]))
    variables = encoder.trainable_variables + decoder.trainable_variables
    gradients = tape.gradient(loss, variables)
    optimizer.apply_gradients(zip(gradients, variables))
    return batch_loss


@tf.function 데코레이터는 훈련 외적인 텐서플로우 연산을 GPU에서 동작하게 해 훈련을 가속할 수 있도록 도와줍니다.

tf.GradientTape()는 학습하며 발생한 모든 연산을 기록하는 테이프입니다. 이것은 모델이 각 스텝의 최종 단계에서 미분값을 구하는 데에 사용

1. Encoder에 소스 문장을 전달해 컨텍스트 벡터인 enc_out 을 생성

2. t=0일 때, Decoder의 Hidden State는 Encoder의 Final State로 정의. h_dec = enc_out[:, -1]
3. Decoder에 입력으로 전달할 'start' 토큰 문장 생성
4. 'start' 문장과 enc_out, Hidden State를 기반으로 다음 단어(t=1)를 예측. pred
5. 예측된 단어와 정답 간의 Loss를 구한 후, t=1의 정답 단어를 다음 입력으로 사용 (예측 단어 X)
6. 반복!

In [37]:
from tqdm import tqdm    # tqdm
import random

EPOCHS = 10

for epoch in range(EPOCHS):
    total_loss = 0
    
    idx_list = list(range(0, kor_tensor.shape[0], BATCH_SIZE))
    random.shuffle(idx_list)
    t = tqdm(idx_list)

    for (batch, idx) in enumerate(t):
        # 마지막 배치가 BATCH_SIZE보다 작은 경우를 처리
        batch_end_idx = min(idx + BATCH_SIZE, kor_tensor.shape[0])
        
        # 배치 슬라이싱
        input_batch = kor_tensor[idx:batch_end_idx]
        target_batch = eng_tensor[idx:batch_end_idx]
        
        # train_step 호출
        batch_loss = train_step(input_batch,
                                target_batch,
                                encoder,
                                decoder,
                                optimizer,
                                eng_tokenizer)
    
        total_loss += batch_loss
        
        t.set_description_str('Epoch %2d' % (epoch + 1))
        t.set_postfix_str('Loss %.4f' % (total_loss.numpy() / (batch + 1)))


Epoch  1:   4%|▍         | 44/987 [01:09<24:49,  1.58s/it, Loss 4.1294]  


ValueError: in user code:

    /tmp/ipykernel_344/3380621219.py:6 train_step  *
        enc_output, enc_hidden = encoder(input, encoder.initialize_hidden_state())
    /tmp/ipykernel_344/3684012298.py:15 call  *
        output, state = self.gru(x, initial_state=hidden)
    /opt/conda/lib/python3.9/site-packages/keras/layers/recurrent.py:716 __call__  **
        return super(RNN, self).__call__(inputs, **kwargs)
    /opt/conda/lib/python3.9/site-packages/keras/engine/base_layer.py:1037 __call__
        outputs = call_fn(inputs, *args, **kwargs)
    /opt/conda/lib/python3.9/site-packages/keras/layers/recurrent_v2.py:443 call
        last_output, outputs, runtime, states = self._defun_gru_call(
    /opt/conda/lib/python3.9/site-packages/keras/layers/recurrent_v2.py:520 _defun_gru_call
        last_output, outputs, new_h, runtime = gru_with_backend_selection(
    /opt/conda/lib/python3.9/site-packages/keras/layers/recurrent_v2.py:820 gru_with_backend_selection
        last_output, outputs, 

In [12]:

# Attention 시각화 함수
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

def plot_attention(attention, sentence, predicted_sentence):
    fig = plt.figure(figsize=(10,10))
    ax = fig.add_subplot(1, 1, 1)
    ax.matshow(attention, cmap='viridis')

    fontdict = {'fontsize': 14}

    ax.set_xticklabels([''] + sentence, fontdict=fontdict, rotation=90)
    ax.set_yticklabels([''] + predicted_sentence, fontdict=fontdict)

    ax.xaxis.set_major_locator(ticker.MultipleLocator(1))
    ax.yaxis.set_major_locator(ticker.MultipleLocator(1))

    plt.show()


In [ ]:
# 번역 함수
def translate(sentence, encoder, decoder):
    result, sentence, attention = evaluate(sentence, encoder, decoder)

    print('Input: %s' % (sentence))
    print('Predicted translation: {}'.format(result))
    
    attention = attention[:len(result.split()), :len(sentence.split())]
    plot_attention(attention, sentence.split(), result.split(' '))



In [ ]:

# 훈련 루프
from tqdm import tqdm
import random
EPOCHS = 10
sample_sentence = "안녕하세요. 너무 피곤해 죽을것같아요!"

for epoch in range(EPOCHS):
    total_loss = 0
    
    idx_list = list(range(0, enc_tensor.shape[0], BATCH_SIZE))
    random.shuffle(idx_list)
    t = tqdm(idx_list)

    for (batch, idx) in enumerate(t):
        batch_loss = train_step(enc_tensor[idx:idx+BATCH_SIZE],
                                dec_tensor[idx:idx+BATCH_SIZE],
                                encoder,
                                decoder,
                                optimizer,
                                dec_tokenizer)
    
        total_loss += batch_loss
        
        t.set_description_str('Epoch %2d' % (epoch + 1))
        t.set_postfix_str('Loss %.4f' % (total_loss.numpy() / (batch + 1)))
    
    # 매 에포크마다 예문을 번역하여 출력
    print(f"\nEpoch {epoch + 1} 번역 결과:")
    translate(sample_sentence, encoder, decoder)

print("훈련 완료!")

In [ ]:
# 주어진 예문
sentences = [
    "오바마는 대통령이다.",
    "시민들은 도시 속에 산다.",
    "커피는 필요 없다.",
    "일곱 명의 사망자가 발생했다."
]

for sentence in sentences:
    translate(sentence, encoder, decoder)